In [3]:
import pandas as pd
import numpy as np
import cPickle as pickle

from sklearn import neighbors
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
prop = pickle.load(open("../../datos/Data.p","rb"))

# Nos quedamos con la parte del set de datos que nos importa

In [5]:
p = prop[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos']]
p.dropna(inplace = True)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Separamos la Latitud-Longitud

In [6]:
p['lat'] = p.apply(lambda row: row[5][0], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
p['lon'] = p.apply(lambda row: row[5][1], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos']]

# Asignamos un valor numerico al tipo de propiedad

In [9]:
def asignarPT( p ):
    if p == 'apartment':
        return 0
    if p == 'house':
        return 1
    if p == 'store':
        return 2
    return 3

In [10]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

# Asignamos un valor a los barrios

In [11]:
def asignarPV(name, dic):
    return dic[name]

In [12]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [13]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)*10
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [14]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [15]:
p = p[['property_type','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos', 'place_value']]

# KNN

In [16]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(p[['property_type','surface_total_in_m2','surface_covered_in_m2',
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos','Cant_LugaresTuristicos',
        'place_value']],p[['price_usd_per_m2']],test_size=0.2)

In [17]:
columnas = ['surface_total_in_m2','surface_covered_in_m2',\
        'Cant_ColeYUniv','Cant_LocalesGastronomicos',\
        'Cant_LugaresTuristicos']

In [27]:
#Parametros modificables de knn
lista_distancias = [1,2] #1 es manhattan, 2 euclidea
lista_n = [x for x in xrange(1,10)]
a = ['auto', 'ball_tree', 'kd_tree', 'brute']
w = ['uniform', 'distance']

In [32]:
res = []
for k in lista_vec:
    for d in lista_distancias:
        #Inicializo la instancia del algoritmo
        knn = neighbors.KNeighborsRegressor(n_neighbors = k, p = d)
        knn.fit(Xtrn, Ytrn['price_usd_per_m2'])
        
        res.append( ( mean_squared_error(Ytest['price_usd_per_m2'], knn.predict(Xtest)), k,d) )

In [33]:
res

[(3219331.1889206013, 1, 1),
 (3306568.5984153715, 1, 2),
 (3131429.6016321178, 2, 1),
 (3147547.2833647728, 2, 2),
 (5735825.4592156149, 3, 1),
 (5782606.2976771854, 3, 2),
 (5929024.1545289923, 4, 1),
 (5974851.9851043178, 4, 2),
 (5157278.8789754212, 5, 1),
 (5199459.5659027118, 5, 2),
 (4431142.4355199384, 6, 1),
 (4418187.8166291192, 6, 2),
 (4286690.5156326247, 7, 1),
 (4325821.1924104616, 7, 2),
 (4045558.0852857246, 8, 1),
 (4092476.1000624872, 8, 2),
 (3883611.324365248, 9, 1),
 (3832639.3253041827, 9, 2)]